In [1]:
import json

In [2]:
shapes = []
smolrange = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 16, 17]
bigrange = smolrange + [19, 23, 31]
for x in bigrange:
    for y in bigrange:
        if y > x:
            continue
        for z in smolrange:
            if z > y:
                continue
            flat = False
            if x > 8 or y > 8 or z > 8:
                flat = True
            if flat and z > 8:
                continue
            shapes.append((x, y, z))


In [3]:
partlist = []
rend_files = {}
obj_files = {}
blockUuids = []

In [4]:
from uuid import uuid4
with open('blockUuidsOfOtherPartSizes.json', 'r') as f:
    existing_uuids = json.load(f)

In [5]:
from pathlib import Path
cellCount = 32
def generate_cube(block):
    return f"""# Blender 4.1.1
# www.blender.org
o CatCube - {block[0]}x{block[1]}x{block[2]}
v {0.5*block[0]}00000 {0.5*block[1]}00000 {-0.5*block[2]}00000
v {0.5*block[0]}00000 {-0.5*block[1]}00000 {-0.5*block[2]}00000
v {0.5*block[0]}00000 {0.5*block[1]}00000 {0.5*block[2]}00000
v {0.5*block[0]}00000 {-0.5*block[1]}00000 {0.5*block[2]}00000
v {-0.5*block[0]}00000 {0.5*block[1]}00000 {-0.5*block[2]}00000
v {-0.5*block[0]}00000 {-0.5*block[1]}00000 {-0.5*block[2]}00000
v {-0.5*block[0]}00000 {0.5*block[1]}00000 {0.5*block[2]}00000
v {-0.5*block[0]}00000 {-0.5*block[1]}00000 {0.5*block[2]}00000
vn -0.0000 1.0000 -0.0000
vn -0.0000 -0.0000 1.0000
vn -1.0000 -0.0000 -0.0000
vn -0.0000 -1.0000 -0.0000
vn 1.0000 -0.0000 -0.0000
vn -0.0000 -0.0000 -1.0000

vt 0.000000 0.000000
vt {block[0]/cellCount} 0.0000000
vt {block[0]/cellCount} {block[2]/cellCount}
vt 0.000000 {block[2]/cellCount}
vt 0.000000 0.000000
vt {block[1]/cellCount} 0.0000000
vt {block[1]/cellCount} {block[0]/cellCount}
vt 0.000000 {block[0]/cellCount}
vt 0.000000 0.000000                   
vt {block[1]/cellCount} 0.0000000 
vt {block[1]/cellCount} {block[2]/cellCount}
vt 0.000000 {block[2]/cellCount}
vt 0.000000 0.000000
vt {block[0]/cellCount} 0.0000000
vt {block[0]/cellCount} {block[2]/cellCount}
vt 0.000000 {block[2]/cellCount}
vt 0.000000 0.000000
vt {block[1]/cellCount} 0.0000000
vt {block[1]/cellCount} {block[2]/cellCount}
vt 0.000000 {block[2]/cellCount}
vt 0.000000 0.000000                   
vt {block[1]/cellCount} 0.0000000 
vt {block[1]/cellCount} {block[0]/cellCount}
vt 0.000000 {block[0]/cellCount}
s 0
usemtl Material
f 1/1/1 5/2/1 7/3/1 3/4/1
f 4/5/2 3/6/2 7/7/2 8/8/2
f 8/9/3 7/10/3 5/11/3 6/12/3
f 6/13/4 2/14/4 4/15/4 8/16/4
f 2/17/5 1/18/5 3/19/5 4/20/5
f 6/21/6 5/22/6 1/23/6 2/24/6
"""

In [6]:
for shape in shapes:
    shape_string = f"{shape[0]}x{shape[1]}x{shape[2]}"
    uuid = existing_uuids.get(shape_string, str(uuid4()))
    existing_uuids[shape_string] = uuid
    part = {
        "box": {
            "x": shape[0],
            "y": shape[1],
            "z": shape[2]
        },
        "name": f"Silicon{shape_string}",
        "physicsMaterial": "Default",
        "renderable": f"$CONTENT_DATA/Renderable/Rends/block{shape_string}.rend",
        "rotationSet": "Default",
        "scripted": {
            "classname": "SiliconBlock",
            "filename": "$CONTENT_DATA/Scripts/silicon/SiliconBlock.lua"
        },
        "showInInventory": False,
        "uuid": uuid,
        "ratings": {
            "density": 2,
            "durability": 3,
            "friction": 5,
            "buoyancy": 2
        },
    }
    partlist.append(part)
    rend_file = {
        "lodList": [
            {
                "mesh": f"$CONTENT_DATA/Renderable/Models/block{shape_string}.obj",
                "subMeshList" : [
                    {
                    "material" : "DifAsgNor",
                    "textureList" : [
                        "$CONTENT_DATA/Renderable/Textures/crab_dif.png",
                        "$CONTENT_DATA/Renderable/Textures/default_asg.png",
                        "$CONTENT_DATA/Renderable/Textures/default_nor.png"
                    ]
                    }
                ]
            }
        ]
    }
    rend_files[f"block{shape_string}.rend"] = rend_file
    obj_file = generate_cube(shape)
    obj_files[f"block{shape_string}.obj"] = obj_file
    blockUuids.append(f"[\"{shape_string}\"] = \"{uuid}\"")

In [7]:
shapeset = {
    "partList": partlist,
}
with open('../Objects/Database/ShapeSets/silicon.shapeset', 'w') as f:
    f.write(json.dumps(shapeset, indent=4))
for filename, content in rend_files.items():
    with open('Rends/'+filename, 'w') as f:
        f.write(json.dumps(content, indent=4))

for filename, content in obj_files.items():
    with open('Models/'+filename, 'w') as f:
        f.write(content)

with open('blockUuids.lua', 'w') as f:
    f.write("sm.MTFastLogic.SiliconBlocksShapeDB.sizeToUuid = {\n")
    for line in blockUuids:
        f.write(f"    {line},\n")
    f.write("}\n")

with open('blockUuidsOfOtherPartSizes.json', 'w') as f:
    f.write(json.dumps(existing_uuids, indent=4))